# Chapter 4 [word2vec 속도 개선]
---
앞의 3장에서 구현한 word2vec은 몇 가지 문제가 있습니다.  
가장 큰 문제는 말뭉치의 어휘 수가 많아지면 계산량도 커진다는 점입니다.  
그래서 이번 장의 목표는 word2vec의 속도 개선입니다.  
구체적으로는 앞 장의 단순한 word2vec에 두 가지 개선을 추가할 겁니다.  
첫 번째 개선으로는 Embedding이라는 새로운 개층을 추가합니다.  
두 번째 개선으로 네거티브 샘플링이라는 새로운 손실 함수를 도입합니다.  
이 두 가지 개선으로 '진짜' word2vec을 완성할 수 있습니다.

## 1. word2vec 개선 ①
앞 장에서 살펴본 CBOW 모델에서 어휘가 100만 개, 은닉층의 뉴런이 100개인 모델을 그려보면 다음과 같을 것입니다.  
<img src=img/fig4-2.png width='500'>  
그림에세 보듯, 입력층과 출력층에는 각 100만 개의 뉴런이 존재합니다.  
이 수많은 뉴런을 계산하는 과정에서 많은 시간이 소요되죠.  
정확히는 다음의 두 계산이 병목이 됩니다.  
* 입력층의 원핫 표현과 가중치 행렬 $W_{in}$의 곱 계산
* 은닉층과 가중치 행렬 $W_{out}$의 곱 및 Softmax 계층의 계산
  
첫 번째로 입력층의 원핫 표현과 관련된 문제입니다.  
단어를 원핫 표현으로 다루기 때문에 어휘 수가 많아지면 원핫 벡터의 크기도 커집니다.  
또한 이 원핫 벡터와 가중치 $W_{in}$을 곱해야 하는데, 이것만으로 계산 자원을 상당히 사용하게 됩니다.  
그래서 이 문제는 Embedding 계층을 도입하는 것으로 해결해 보겠습니다.  

두 번째 문제는 은닉층 이후의 계산입니다. 우선 은닉층과 가중치 행렬 $W_{out}$의 곱만 해도 계산량이 상당합니다.  
그리고 Softmax 계층에서도 다루는 어휘가 많아짐에 따라 계산량이 증가하는 문제가 있습니다.  
이 문제는 네거티브 샘플링이라는 새로운 손실 함수를 도입해 해결해 보겠습니다.

### 1.1 Embedding 계층
<img src=img/fig4-3.png width='500'>  
위 그림은 어휘 수가 100만 개인 경우를 가정했을 때의 MatMul 계층의 행렬 곱입니다.  
결과적으로 수행하는 일은 단지 행렬의 특정 행을 추출하는 것뿐입니다.  
따라서 원핫 표현으로의 변환과 MatMul 계층의 행렬 곱 계산은 사실 필요가 없습니다.  

그러면 가중치 매개변수로부터 '단어 ID에 해당하는 행(벡터)'을 추출하는 계층을 만들고, 그 계층을 Embedding 계층이라고 부르겠습니다.  

### 1.2 Embedding 계층 구현
가중치로부터 특정 행을 추출할 땐 W[2]나 W[5]처럼 특정 행을 명시해주면 됩니다.

In [1]:
import numpy as np
W = np.arange(21).reshape(7, 3)
print(W)
print("-"*50)
print("W[2] : ", W[2])
print("W[5] : ", W[5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
--------------------------------------------------
W[2] :  [6 7 8]
W[5] :  [15 16 17]


또한, 가중치 W로부터 여러 행을 한꺼번에 추출하는 일도 간단하게 할 수 있습니다.  
원하는 행 번호들을 배열에 면시하기만 하면 됩니다.

In [2]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

이제 Embedding 계층의 forward() 메서드를 구현하겠습니다.  

In [3]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

이어서 역전파(backward)를 구현하겠습니다.  
역전파를 구현할 땐 한가지 주의할 점이 있습니다.  
바로 idx의 원소가 중복될 때 발생합니다. 예컨대 idx가 [0, 2, 0, 4]일 경우를 생각해봅시다.  
<img src=img/fig4-5.png width='500'>  

위 그림과 같이 dh의 각 행 값을 idx가 가리키는 장소에 할당해보겠습니다.  
그러면 dW의 0번째 행에 2개의 값이 할당됩니다. 먼저 쓰여진 값을 덮어쓴다는 뜻이죠.  

이 중복 문제를 해결하려면 '할당'이 아닌 '더하기'를 해야 합니다.

In [4]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        '''
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        '''
            
        np.add.at(dW, self.idx, dout)
        
        return None

위 구현에서는 for 문을 사용했는데, np.add.at()을 사용해도 됩니다.  
np.add.at(A, idx, B)는 B를 A의 idx번째 행에 더해줍니다.  
일반적으로 파이썬에서 for 문보다는 넘파이의 내장 메서드를 사용하는 편이 더 빠릅니다.  
넘파이의 메서드에는 속도와 효율을 높여주는 최적화가 적용돼 있기 때문입니다.  

## 2. word2vec 개선 ②
두 번째 개선의 목표는 은닉층 이후의 행렬 곱과 Softmax 계층의 계산의 병목 해소입니다.  
문제를 해결하기 위해서 **네거티브 샘플링**(부정적 샘플링)이라는 기법을 사용합니다.  
Softmax 대신 네거티브 샘플링을 이용하면 어휘가 아무리 많아져도 계산량을 낮은 수준에서 일정하게 억제할 수 있습니다.  

### 2.1 은닉층 이후 계산의 문제점
<img src=img/fig4-6.png width='500'>  
그림에서 보듯, 은닉층 이후 계산이 오래 걸리는 곳은 다음 두 부분입니다.  
* 은닉층의 뉴런과 가중치 행렬($W_{out}$)의 곱
* Softmax 계층의 계산
  
첫 번째는 거대한 행렬을 곱하는 문제입니다.
은닉층의 벡터 크기가 100이고, 가중치 행렬의 크기가 100x100만입니다.  
이렇게 큰 행렬의 곱을 계산하려면 시간이 오래걸리고 메모리도 많이 필요합니다.  

두 번째로, Softmax에서도 같은 문제가 발생합니다.  
어휘가 많아지면 Softmax의 계산량도 증가합니다.  
$$ y_k=\frac{exp(s_k)}{\sum_{i=1}^{1000000} exp(s_i)} $$

Softmax에서 분모의 값을 얻으려면 exp 계산을 100만 번 수행해야합니다.  
이 계산도 어휘 수에 비례해 증가하므로 Softmax를 대신할 '가벼운' 계산이 절실합니다.  

### 2.2 다중 분류에서 이진 분류로
네거티브 샘플링 기법의 핵심 아이디어는 '이진 분류<sup>binary classification</sup>'에 있습니다.  
더 정확하게 말하면, '다중 분류<sup>multi-class classification</sup>'를 '이진 분류'로 근사하는 것이 네거티브 샘플링을 이해하는데 중요한 포인트입니다.  

지금까지 우리는 '다중 분류'문제를 다뤄왔습니다. 바로 앞의 예도 100만 개의 단어 중에서 옳은 단어 하나를 선택하는 문제였습니다.  
그렇다면 이러한 문제를 '이진 분류'문제로 다룰 수는 없을까요?  
더 정확하게 말하면, '다중 분류'문제를 '이진 분류'문제로 근사할 수는 없을까요?  

예를들어 맥락으로 'you'와 'goodbey'를 주었을 때, 타깃(정답)이 'say'가 맞는지를 물어보는 겁니다.  
<img src=img/fig4-7.png width='500'>
<img src=img/fig4-8.png width='500'> 

그림처럼 출력 측의 가중치 $W_{out}$에서는 각 단어 ID의 단어 벡터가 각각의 열로 저장되어 있습니다.  
이 예에서는 "say"에 해당하는 단어 벡터를 추출합니다. 그리고 그 벡터와 은닉층 뉴런과의 내적을 구합니다.  
이렇게 구한 값이 최종 점수인 것이죠.  

---
이전까지의 출력층에서는 모든 단어를 대상으로 계산을 수행했습니다.  
하지만 여기에서는 "say"라는 단어 하나에 주목하여 그 점수만을 계산하는 게 차이입니다.  
그리고 시그모이드 함수를 이용해 그 점수를 확률로 변환합니다.

### 2.3 시그모이드 함수와 교차 엔트로피 오차
이진 분류 문제를 신경망으로 풀려면 점수에 시그모이드 함수를 적용해 확률로 변환합니다.  
그리고 손실을 구할 때는 손실 함수로 '교차 엔트로피 오차'를 사용합니다.  
이 조합은 역전파때 미분값이 깔끔하게 떨어지므로 신경망에서 자주 사용됩니다.  
<img src=img/fig4-10.png width='600'>   
위 그림에서 y는 신경망이 출력한 확률이고, t는 정답 레이블입니다.  
y가 1(100%)에 가까워질수록 오차가 줄어든다는 뜻입니다.  
반대로 y가 1로부터 멀어지면 오차가 커집니다.  
그리고 그 오차가 앞 계층으로 흘러가므로, 오차가 크면 '크게' 학습하고, 작으면 '작게' 학습하게 됩니다.

---
* $\frac{\partial L}{\partial x}$이 $y-t$로 유도되는 과정  

$\frac{\partial L}{\partial x}=\frac{\partial L}{\partial y}\frac{\partial y}{\partial x}$  
$\frac{\partial L}{\partial y}=-\frac{t}{y}+\frac{1-t}{1-y}=\frac{y-t}{y(1-y)}$  
$\frac{\partial y}{\partial x}=y(1-y)$  

따라서 $\frac{\partial L}{\partial x}=\frac{y-t}{y(1-y)}\frac{y(1-y)}{1}=y-t$가 됩니다.

### 2.4 다중 분류에서 이진 분류로 (구현)
이진 분류를 수행하는 word2vec의 전체 모습을 보겠습니다.  
단어 ID는 'you' = 0 / 'say' = 1 / 'goodbey' = 2 라고 가정했습니다.  
<img src=img/fig4-12.png width='600'>   
여기에서는 은닉층 뉴런 h와, 출력 측의 가중치 $W_{out}$에서 단어 "say"에 해당하는 단어 벡터와의 내적을 계산합니다.  
그리고 Sigmoid with Loss 계층에 입력해 최종 손실을 얻습니다.  

이 부분을 단순하게 만들어서 Embedding Dot 계층을 만들겠습니다.  
<img src=img/fig4-13.png width='600'>   

In [5]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 2.5 네거티브 샘플링
이제 '다중 분류'에서 '이진 분류'로 변환함으로써 병목현상은 해결했습니다.  
하지만 아직 해야 할 일이 하나 남았습니다.  
지금까지는 긍정적인 예(정답)에 대해서만 학습했습니다.  
그래서 부정적인 예(오답)를 입력했을 때 어떤 결과가 나올지 확실하지 않습니다.  
<img src=img/fig4-15.png width='600'>  
위 그림처럼 긍정적 예(정답)인 "say"에 대해서만 학습했습니다.  
그래서 부정적인 예("say" 이외의 단어)에 대해서는 아무것도 학습하지 않았습니다.  
따라서 우리가 해야 할 일은 긍정적 예("say")에 대해서는 Sigmoid 계층의 출력을 1에 가깝게 만들고, 부정적 예("say" 이외의 단어)에 대해서는 Sigmoid 계층의 출력을 0에 가깝게 만드는 것입니다.  
<img src=img/fig4-16.png width='600'>  
그렇다고 모든 부정적 예를 학습하면 계산량이 다시 증가하기 때문에 근사할 수 있는 해법이 필요합니다.  
바로 부정적 예를 몇 개(5개라든지, 10개라든지) 선택하여 학습합니다.  
즉, 적은 수의 부정적 예를 샘플링해 사용합니다. 이것이 바로 '네거티브 샘플링' 기법이 의미하는 바입니다.  
<img src=img/fig4-17.png width='600'>  

### 2.6 네거티브 샘플링의 샘플링 기법
샘플링을 할 때 어떻게 하는 것이 좋을까요?  
가장 쉬운 방법은 무작위로 선택하는 것일 겁니다.  
하지만 무작위로 선택했을 때 우연히도 '희소한 단어'만 선택되었다면 어떻게 될까요?  
당연히 결과가 좋지 않을 것입니다. 왜냐하면 실전 문제에서도 희소한 단어는 거의 출현하지 않기 때문이죠.  
그래서 흔한 단어를 잘 처리하는 것이 좋은 결과로 이어질 것입니다.  

이런 무작위 샘플링 보다 좋은 방법이 있습니다.  
바로 말뭉치의 통계 데이터를 기초로 샘플링하는 방법입니다.  
말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 희소 단어를 적게 추출하는 것입니다.  
말뭉치에서의 단어 빈도를 기준으로 샘플링하려면, 먼저 말뭉치에서 각 단어의 출현 횟수를 구해 '확률분포'로 나타냅니다.  
그런 다음 그 확률분포대로 단어를 샘플링하면 됩니다.  
<img src=img/fig4-18.png width='600'>  

In [6]:
import numpy as np

words = ['you', 'say', 'goodbey', 'I', 'hello', '.']

#5개만 무작위로 샘플링(중복 있음)
sample = np.random.choice(words, size=5)
print(sample)

#5개만 무작위로 샘플링(중복 없음)
sample = np.random.choice(words, size=5, replace=False)
print(sample)

#확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
sample = np.random.choice(words, p=p)
print(sample)

['.' 'say' 'I' 'goodbey' 'I']
['hello' 'goodbey' 'you' 'I' 'say']
I


np.random.choice()는 무작위 샘플링 용도로 이용할 수 있습니다.  
size를 지정해 주면 샘플링을 size 만큼 수행합니다.  
또 인수에 replace=False를 지정하면 샘플링 시 중복을 없애줍니다.  
그리고 인수 p에 확률분포를 담은 리스트를 지정하면 그 확률분포대로 샘플링을 해줍니다.  
이제 이 함수를 사용해 부정적 예를 샘플링하기만 하면 됩니다.  

다만 word2vec의 네거티브 샘플링에서는 앞의 확률분포에서 한 가지를 수정하라고 권고하고 있습니다.  
바로 아래 식 처럼 확률분포에 0.75를 제곱하는 것입니다.  
$$ P'(w_i)=\frac{P(w_i)^{0.75}}{\sum_j^nP(w_j)^{0.75}} $$  
왜 이렇게 하는 걸까요? 바로 출현 확률이 낮은 단어를 버리지 않기 위해서입니다.  
0.75 제곱을 함으로써, 원래 확률이 낮은 단어의 확률을 살짝 높일 수 있습니다.  

In [7]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


지금까지 살펴본 샘플링 방식을 UnigramSampler라는 이름으로 구현했습니다.

In [8]:
#import sys
#sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample
    
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[2 4]
 [1 4]
 [1 2]]


### 2.7 네거티브 샘플링 구현
지금까지 종합하여 NegativeSamplelingLoss 계층으로 구현했습니다.

In [9]:
#import sys
#sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections


class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh


class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample


class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

## 3. 개선판 word2vec 학습

### 3.1 CBOW 모델 구현
Embedding 계층과 Negative Sampleling Loss 계층을 적용하여 구현했습니다.

In [10]:
#import sys
#sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

### 3.2 CBOW 모델 학습 코드

In [1]:
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import create_contexts_target, to_cpu, to_gpu
from common.cbow import CBOW
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

------------------------------------------------------------
                       GPU Mode (cupy)
------------------------------------------------------------

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 0[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 1[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 1[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 1[s] | 손실 3.93
| 에폭 1 |  반복 121 / 9295 | 시간 1[s] | 손실 3.79
| 에폭 1 |  반복 141 / 9295 | 시간 2[s] | 손실 3.63
| 에폭 1 |  반복 161 / 9295 | 시간 2[s] | 손실 3.50
| 에폭 1 |  반복 181 / 9295 | 시간 2[s] | 손실 3.37
| 에폭 1 |  반복 201 / 9295 | 시간 3[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 3[s] | 손실 3.18
| 에폭 1 |  반복 241 / 9295 | 시간 3[s] | 손실 3.10
| 에폭 1 |  반복 261 / 9295 | 시간 4[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 4[s] | 손실 2.99
| 에폭 1 |  반복 301 / 9295 | 시간 4[s] | 손실 2.93
| 에폭 1 |  반복 321 / 9295 | 시간 4[s] | 손실 2.88
| 에폭 1 |  반복 341 / 9295 | 시간 5[s] | 손실 2.87
| 에폭 1 |  반복 361 / 9295 | 시간 5[s] | 손실 2.83
| 에폭 1 |

| 에폭 1 |  반복 3521 / 9295 | 시간 52[s] | 손실 2.45
| 에폭 1 |  반복 3541 / 9295 | 시간 52[s] | 손실 2.42
| 에폭 1 |  반복 3561 / 9295 | 시간 53[s] | 손실 2.44
| 에폭 1 |  반복 3581 / 9295 | 시간 53[s] | 손실 2.41
| 에폭 1 |  반복 3601 / 9295 | 시간 53[s] | 손실 2.44
| 에폭 1 |  반복 3621 / 9295 | 시간 54[s] | 손실 2.42
| 에폭 1 |  반복 3641 / 9295 | 시간 54[s] | 손실 2.45
| 에폭 1 |  반복 3661 / 9295 | 시간 54[s] | 손실 2.45
| 에폭 1 |  반복 3681 / 9295 | 시간 54[s] | 손실 2.45
| 에폭 1 |  반복 3701 / 9295 | 시간 55[s] | 손실 2.42
| 에폭 1 |  반복 3721 / 9295 | 시간 55[s] | 손실 2.41
| 에폭 1 |  반복 3741 / 9295 | 시간 55[s] | 손실 2.42
| 에폭 1 |  반복 3761 / 9295 | 시간 56[s] | 손실 2.39
| 에폭 1 |  반복 3781 / 9295 | 시간 56[s] | 손실 2.42
| 에폭 1 |  반복 3801 / 9295 | 시간 56[s] | 손실 2.42
| 에폭 1 |  반복 3821 / 9295 | 시간 57[s] | 손실 2.44
| 에폭 1 |  반복 3841 / 9295 | 시간 57[s] | 손실 2.43
| 에폭 1 |  반복 3861 / 9295 | 시간 57[s] | 손실 2.42
| 에폭 1 |  반복 3881 / 9295 | 시간 58[s] | 손실 2.42
| 에폭 1 |  반복 3901 / 9295 | 시간 58[s] | 손실 2.43
| 에폭 1 |  반복 3921 / 9295 | 시간 58[s] | 손실 2.43
| 에폭 1 |  반복 3941 / 9295 | 시간 58[s

| 에폭 1 |  반복 7081 / 9295 | 시간 106[s] | 손실 2.29
| 에폭 1 |  반복 7101 / 9295 | 시간 106[s] | 손실 2.29
| 에폭 1 |  반복 7121 / 9295 | 시간 106[s] | 손실 2.29
| 에폭 1 |  반복 7141 / 9295 | 시간 107[s] | 손실 2.31
| 에폭 1 |  반복 7161 / 9295 | 시간 107[s] | 손실 2.32
| 에폭 1 |  반복 7181 / 9295 | 시간 107[s] | 손실 2.29
| 에폭 1 |  반복 7201 / 9295 | 시간 107[s] | 손실 2.28
| 에폭 1 |  반복 7221 / 9295 | 시간 108[s] | 손실 2.29
| 에폭 1 |  반복 7241 / 9295 | 시간 108[s] | 손실 2.29
| 에폭 1 |  반복 7261 / 9295 | 시간 108[s] | 손실 2.32
| 에폭 1 |  반복 7281 / 9295 | 시간 109[s] | 손실 2.30
| 에폭 1 |  반복 7301 / 9295 | 시간 109[s] | 손실 2.28
| 에폭 1 |  반복 7321 / 9295 | 시간 109[s] | 손실 2.29
| 에폭 1 |  반복 7341 / 9295 | 시간 110[s] | 손실 2.28
| 에폭 1 |  반복 7361 / 9295 | 시간 110[s] | 손실 2.32
| 에폭 1 |  반복 7381 / 9295 | 시간 110[s] | 손실 2.29
| 에폭 1 |  반복 7401 / 9295 | 시간 110[s] | 손실 2.27
| 에폭 1 |  반복 7421 / 9295 | 시간 111[s] | 손실 2.30
| 에폭 1 |  반복 7441 / 9295 | 시간 111[s] | 손실 2.25
| 에폭 1 |  반복 7461 / 9295 | 시간 111[s] | 손실 2.31
| 에폭 1 |  반복 7481 / 9295 | 시간 112[s] | 손실 2.28
| 에폭 1 |  반복 

| 에폭 2 |  반복 1301 / 9295 | 시간 158[s] | 손실 2.17
| 에폭 2 |  반복 1321 / 9295 | 시간 159[s] | 손실 2.16
| 에폭 2 |  반복 1341 / 9295 | 시간 159[s] | 손실 2.18
| 에폭 2 |  반복 1361 / 9295 | 시간 159[s] | 손실 2.18
| 에폭 2 |  반복 1381 / 9295 | 시간 160[s] | 손실 2.16
| 에폭 2 |  반복 1401 / 9295 | 시간 160[s] | 손실 2.18
| 에폭 2 |  반복 1421 / 9295 | 시간 160[s] | 손실 2.18
| 에폭 2 |  반복 1441 / 9295 | 시간 160[s] | 손실 2.16
| 에폭 2 |  반복 1461 / 9295 | 시간 161[s] | 손실 2.19
| 에폭 2 |  반복 1481 / 9295 | 시간 161[s] | 손실 2.13
| 에폭 2 |  반복 1501 / 9295 | 시간 161[s] | 손실 2.16
| 에폭 2 |  반복 1521 / 9295 | 시간 162[s] | 손실 2.15
| 에폭 2 |  반복 1541 / 9295 | 시간 162[s] | 손실 2.15
| 에폭 2 |  반복 1561 / 9295 | 시간 162[s] | 손실 2.14
| 에폭 2 |  반복 1581 / 9295 | 시간 163[s] | 손실 2.14
| 에폭 2 |  반복 1601 / 9295 | 시간 163[s] | 손실 2.16
| 에폭 2 |  반복 1621 / 9295 | 시간 163[s] | 손실 2.16
| 에폭 2 |  반복 1641 / 9295 | 시간 164[s] | 손실 2.21
| 에폭 2 |  반복 1661 / 9295 | 시간 164[s] | 손실 2.17
| 에폭 2 |  반복 1681 / 9295 | 시간 164[s] | 손실 2.12
| 에폭 2 |  반복 1701 / 9295 | 시간 164[s] | 손실 2.16
| 에폭 2 |  반복 

| 에폭 2 |  반복 4801 / 9295 | 시간 211[s] | 손실 2.10
| 에폭 2 |  반복 4821 / 9295 | 시간 211[s] | 손실 2.11
| 에폭 2 |  반복 4841 / 9295 | 시간 212[s] | 손실 2.12
| 에폭 2 |  반복 4861 / 9295 | 시간 212[s] | 손실 2.11
| 에폭 2 |  반복 4881 / 9295 | 시간 212[s] | 손실 2.11
| 에폭 2 |  반복 4901 / 9295 | 시간 212[s] | 손실 2.13
| 에폭 2 |  반복 4921 / 9295 | 시간 213[s] | 손실 2.13
| 에폭 2 |  반복 4941 / 9295 | 시간 213[s] | 손실 2.09
| 에폭 2 |  반복 4961 / 9295 | 시간 213[s] | 손실 2.09
| 에폭 2 |  반복 4981 / 9295 | 시간 214[s] | 손실 2.04
| 에폭 2 |  반복 5001 / 9295 | 시간 214[s] | 손실 2.10
| 에폭 2 |  반복 5021 / 9295 | 시간 214[s] | 손실 2.12
| 에폭 2 |  반복 5041 / 9295 | 시간 215[s] | 손실 2.10
| 에폭 2 |  반복 5061 / 9295 | 시간 215[s] | 손실 2.10
| 에폭 2 |  반복 5081 / 9295 | 시간 215[s] | 손실 2.13
| 에폭 2 |  반복 5101 / 9295 | 시간 215[s] | 손실 2.10
| 에폭 2 |  반복 5121 / 9295 | 시간 216[s] | 손실 2.12
| 에폭 2 |  반복 5141 / 9295 | 시간 216[s] | 손실 2.11
| 에폭 2 |  반복 5161 / 9295 | 시간 216[s] | 손실 2.12
| 에폭 2 |  반복 5181 / 9295 | 시간 217[s] | 손실 2.10
| 에폭 2 |  반복 5201 / 9295 | 시간 217[s] | 손실 2.07
| 에폭 2 |  반복 

...

| 에폭 9 |  반복 6001 / 9295 | 시간 1211[s] | 손실 1.48
| 에폭 9 |  반복 6021 / 9295 | 시간 1212[s] | 손실 1.52
| 에폭 9 |  반복 6041 / 9295 | 시간 1212[s] | 손실 1.52
| 에폭 9 |  반복 6061 / 9295 | 시간 1212[s] | 손실 1.50
| 에폭 9 |  반복 6081 / 9295 | 시간 1213[s] | 손실 1.51
| 에폭 9 |  반복 6101 / 9295 | 시간 1213[s] | 손실 1.51
| 에폭 9 |  반복 6121 / 9295 | 시간 1213[s] | 손실 1.48
| 에폭 9 |  반복 6141 / 9295 | 시간 1214[s] | 손실 1.46
| 에폭 9 |  반복 6161 / 9295 | 시간 1214[s] | 손실 1.52
| 에폭 9 |  반복 6181 / 9295 | 시간 1214[s] | 손실 1.49
| 에폭 9 |  반복 6201 / 9295 | 시간 1214[s] | 손실 1.50
| 에폭 9 |  반복 6221 / 9295 | 시간 1215[s] | 손실 1.49
| 에폭 9 |  반복 6241 / 9295 | 시간 1215[s] | 손실 1.51
| 에폭 9 |  반복 6261 / 9295 | 시간 1215[s] | 손실 1.52
| 에폭 9 |  반복 6281 / 9295 | 시간 1216[s] | 손실 1.50
| 에폭 9 |  반복 6301 / 9295 | 시간 1216[s] | 손실 1.49
| 에폭 9 |  반복 6321 / 9295 | 시간 1216[s] | 손실 1.50
| 에폭 9 |  반복 6341 / 9295 | 시간 1217[s] | 손실 1.52
| 에폭 9 |  반복 6361 / 9295 | 시간 1217[s] | 손실 1.55
| 에폭 9 |  반복 6381 / 9295 | 시간 1217[s] | 손실 1.53
| 에폭 9 |  반복 6401 / 9295 | 시간 1217[s] | 

| 에폭 10 |  반복 121 / 9295 | 시간 1263[s] | 손실 1.42
| 에폭 10 |  반복 141 / 9295 | 시간 1263[s] | 손실 1.44
| 에폭 10 |  반복 161 / 9295 | 시간 1263[s] | 손실 1.44
| 에폭 10 |  반복 181 / 9295 | 시간 1264[s] | 손실 1.46
| 에폭 10 |  반복 201 / 9295 | 시간 1264[s] | 손실 1.45
| 에폭 10 |  반복 221 / 9295 | 시간 1264[s] | 손실 1.41
| 에폭 10 |  반복 241 / 9295 | 시간 1265[s] | 손실 1.41
| 에폭 10 |  반복 261 / 9295 | 시간 1265[s] | 손실 1.44
| 에폭 10 |  반복 281 / 9295 | 시간 1265[s] | 손실 1.41
| 에폭 10 |  반복 301 / 9295 | 시간 1265[s] | 손실 1.46
| 에폭 10 |  반복 321 / 9295 | 시간 1266[s] | 손실 1.51
| 에폭 10 |  반복 341 / 9295 | 시간 1266[s] | 손실 1.42
| 에폭 10 |  반복 361 / 9295 | 시간 1266[s] | 손실 1.45
| 에폭 10 |  반복 381 / 9295 | 시간 1267[s] | 손실 1.43
| 에폭 10 |  반복 401 / 9295 | 시간 1267[s] | 손실 1.49
| 에폭 10 |  반복 421 / 9295 | 시간 1267[s] | 손실 1.45
| 에폭 10 |  반복 441 / 9295 | 시간 1268[s] | 손실 1.48
| 에폭 10 |  반복 461 / 9295 | 시간 1268[s] | 손실 1.46
| 에폭 10 |  반복 481 / 9295 | 시간 1268[s] | 손실 1.44
| 에폭 10 |  반복 501 / 9295 | 시간 1269[s] | 손실 1.46
| 에폭 10 |  반복 521 / 9295 | 시간 1269[s] | 

| 에폭 10 |  반복 3501 / 9295 | 시간 1314[s] | 손실 1.46
| 에폭 10 |  반복 3521 / 9295 | 시간 1314[s] | 손실 1.46
| 에폭 10 |  반복 3541 / 9295 | 시간 1315[s] | 손실 1.46
| 에폭 10 |  반복 3561 / 9295 | 시간 1315[s] | 손실 1.47
| 에폭 10 |  반복 3581 / 9295 | 시간 1315[s] | 손실 1.45
| 에폭 10 |  반복 3601 / 9295 | 시간 1316[s] | 손실 1.47
| 에폭 10 |  반복 3621 / 9295 | 시간 1316[s] | 손실 1.47
| 에폭 10 |  반복 3641 / 9295 | 시간 1316[s] | 손실 1.43
| 에폭 10 |  반복 3661 / 9295 | 시간 1316[s] | 손실 1.49
| 에폭 10 |  반복 3681 / 9295 | 시간 1317[s] | 손실 1.50
| 에폭 10 |  반복 3701 / 9295 | 시간 1317[s] | 손실 1.47
| 에폭 10 |  반복 3721 / 9295 | 시간 1317[s] | 손실 1.43
| 에폭 10 |  반복 3741 / 9295 | 시간 1318[s] | 손실 1.46
| 에폭 10 |  반복 3761 / 9295 | 시간 1318[s] | 손실 1.45
| 에폭 10 |  반복 3781 / 9295 | 시간 1318[s] | 손실 1.48
| 에폭 10 |  반복 3801 / 9295 | 시간 1319[s] | 손실 1.49
| 에폭 10 |  반복 3821 / 9295 | 시간 1319[s] | 손실 1.46
| 에폭 10 |  반복 3841 / 9295 | 시간 1319[s] | 손실 1.45
| 에폭 10 |  반복 3861 / 9295 | 시간 1319[s] | 손실 1.44
| 에폭 10 |  반복 3881 / 9295 | 시간 1320[s] | 손실 1.42
| 에폭 10 |  반복 3901 /

| 에폭 10 |  반복 6861 / 9295 | 시간 1365[s] | 손실 1.43
| 에폭 10 |  반복 6881 / 9295 | 시간 1365[s] | 손실 1.47
| 에폭 10 |  반복 6901 / 9295 | 시간 1366[s] | 손실 1.42
| 에폭 10 |  반복 6921 / 9295 | 시간 1366[s] | 손실 1.49
| 에폭 10 |  반복 6941 / 9295 | 시간 1366[s] | 손실 1.48
| 에폭 10 |  반복 6961 / 9295 | 시간 1366[s] | 손실 1.49
| 에폭 10 |  반복 6981 / 9295 | 시간 1367[s] | 손실 1.45
| 에폭 10 |  반복 7001 / 9295 | 시간 1367[s] | 손실 1.48
| 에폭 10 |  반복 7021 / 9295 | 시간 1367[s] | 손실 1.49
| 에폭 10 |  반복 7041 / 9295 | 시간 1368[s] | 손실 1.48
| 에폭 10 |  반복 7061 / 9295 | 시간 1368[s] | 손실 1.47
| 에폭 10 |  반복 7081 / 9295 | 시간 1368[s] | 손실 1.49
| 에폭 10 |  반복 7101 / 9295 | 시간 1369[s] | 손실 1.45
| 에폭 10 |  반복 7121 / 9295 | 시간 1369[s] | 손실 1.49
| 에폭 10 |  반복 7141 / 9295 | 시간 1369[s] | 손실 1.46
| 에폭 10 |  반복 7161 / 9295 | 시간 1369[s] | 손실 1.49
| 에폭 10 |  반복 7181 / 9295 | 시간 1370[s] | 손실 1.50
| 에폭 10 |  반복 7201 / 9295 | 시간 1370[s] | 손실 1.48
| 에폭 10 |  반복 7221 / 9295 | 시간 1370[s] | 손실 1.50
| 에폭 10 |  반복 7241 / 9295 | 시간 1371[s] | 손실 1.46
| 에폭 10 |  반복 7261 /

<Figure size 640x480 with 1 Axes>

### 3.3 CBOW 모델 평가

In [1]:
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle
import numpy as np
from common import config
config.GPU = False

pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)


[query] you
 we: 0.75927734375
 i: 0.71240234375
 your: 0.6181640625
 they: 0.58349609375
 anything: 0.5810546875

[query] year
 month: 0.86572265625
 spring: 0.7880859375
 summer: 0.783203125
 week: 0.7578125
 decade: 0.69189453125

[query] car
 truck: 0.62060546875
 auto: 0.59912109375
 luxury: 0.5791015625
 cars: 0.5615234375
 window: 0.5390625

[query] toyota
 honda: 0.6513671875
 mazda: 0.63720703125
 minicomputers: 0.61669921875
 coated: 0.6064453125
 seita: 0.60595703125
--------------------------------------------------

[analogy] king:man = queen:?
 a.m: 6.265625
 woman: 5.78125
 husband: 5.10546875
 mother: 4.87109375
 naczelnik: 4.79296875

[analogy] take:took = go:?
 're: 4.69140625
 were: 4.6171875
 a.m: 4.5234375
 hour: 4.23046875
 came: 4.1796875

[analogy] car:cars = child:?
 a.m: 7.0234375
 rape: 5.51953125
 children: 5.48828125
 daffynition: 5.08203125
 males: 4.6796875

[analogy] good:better = bad:?
 rather: 6.125
 less: 5.7109375
 more: 5.6484375
 greater: 4.804687